## Workflow:
- Import Dependencies
- Download Dataset
- Read file
- Process Text
- Creating training and target examples
- Creating training batches
- Build model
- Trying model
- Creating Checkpoints
- Executing Training
- Export Generator

### Import Dependencies

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

### Download Shakespeare Dataset

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 2s 2us/step


### Read the File

In [8]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode('utf-8')  # a string
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [9]:
text[:250]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n'

In [36]:
# The unique characters in the file
vocab = sorted(set(text))  # List of characters
print(f'{len(vocab)} unique characters')

65 unique characters


### Process the text

In [37]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=vocab, mask_token=None)  # This returns a tokenizer(character level-as our vocab is just characters!)

In [ ]:
# char_from_ids object is used to convert ids to characters
# Notice that we gave it the same reference vocab as we gave to ids_from_chars object
# This is because we want the UNK token to be set the same way
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [69]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [13]:
ids = ids_from_chars(chars)  # Generating numeric sequence
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [41]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [45]:
# tf.strings.reduce_join to join the characters back into strings. 
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [46]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Creating training and target examples

In [47]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1], dtype=int64)>

In [49]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
ids_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [68]:
for ids in ids_dataset.take(5):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t


In [56]:
seq_length = 100

In [57]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [62]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [63]:
# Takes in a string, returns input and target texts (We'll map this func. to our dataset object!)
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [64]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [65]:
dataset = sequences.map(split_input_target)

In [66]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### Creating training batches

In [67]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### Build The Model

In [70]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [71]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, # we'll get output y at each timestamp through the Dense layer
                                   return_state=True)  # Every gru layer passes hidden activations to next layer
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [72]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### Trying the Model

In [73]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [74]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [75]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [76]:
sampled_indices

array([31,  2, 34, 44, 62, 47, 49, 38, 19, 33,  1, 40, 65, 18, 28, 59, 38,
       22, 35, 50,  9, 47,  1, 46, 42, 64, 53,  1, 25, 16, 26, 14, 27, 37,
       52, 55, 25, 62, 38, 22, 23, 43, 16,  4, 46, 56,  1, 32, 24, 18,  6,
       55, 41, 11, 44,  3,  4, 19, 14, 18,  7,  0, 34, 48, 36, 40, 39, 48,
       36, 10,  7, 16, 25, 22, 35, 52, 39, 21, 55,  4, 25, 42, 37, 46, 51,
       62, 12,  0, 60, 42, 29, 29, 15, 30,  1, 12, 11, 23, 55, 34],
      dtype=int64)

In [77]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"dness! Make not impossible\nThat which but seems unlike: 'tis not impossible\nBut one, the wicked'st c"

Next Char Predictions:
 b"R UewhjYFT\nazEOtYIVk.h\ngcyn\nLCMANXmpLwYIJdC$gq\nSKE'pb:e!$FAE,[UNK]UiWaZiW3,CLIVmZHp$LcXglw;[UNK]ucPPBQ\n;:JpU"


### Train the Model

In [79]:
# Since the model returns logits, make the from_logits flag set to true.
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True) 

In [80]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1894026, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [81]:
tf.exp(example_batch_mean_loss).numpy()

65.98336

In [82]:
model.compile(optimizer='adam', loss=loss)

### Creating Checkpoints

In [83]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [84]:
EPOCHS = 5

In [85]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
172/172 [==============================] - 608s 4s/step - loss: 2.7193
Epoch 2/5
172/172 [==============================] - 14978s 88s/step - loss: 1.9818
Epoch 3/5
172/172 [==============================] - 468s 3s/step - loss: 1.7016
Epoch 4/5
172/172 [==============================] - 457s 3s/step - loss: 1.5420
Epoch 5/5
172/172 [==============================] - 460s 3s/step - loss: 1.4441


### Generate text

In [86]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [87]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [88]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
-this is a preaser tomple shall prove me, ord my grie writnes;
Come, in thire and myself:
I am again mines;
And then know father
Or doathance I'll bear this, tear'st.

AmmI:
Sir, I worr they, thereword won;
Harry you said---Op do quickle your devorm,
And may I can past to them,
If not in mine dow, it zadisht
that thou heart your hands I pretty,
'Tis proud, 'Dile no your it on the queen!

POMP:
A since to your seaved's house in the dial 'goo?

POLIXENES:
I may came feroul of thy nearts of morture
But head as favour tailt.

NORTHUMBERLAND:
Then, madam, oney.

FRIAR LAURENCE:
Eecle him distance
To citize-sinking shile; chalge thy dead.

ISABELLA:
Hat scand it sweet rage the towards.

PETER:
I printers and delay'd.' Hanton: think to more out.
Must is every blood want;
What I cannot him, in hotest part his art to wink.

KING EDWARD II:
Which he shall still fetch forfurred;
His a foush tormers from the day,
You are un and holse victor.

KATHARINA:
I'll to you the duke of king?

BULINT

If you want the model to generate text faster the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [90]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nChallow, play. What lay in the city Coriolanumina, 'Tis dispade\nAnd tell our darnenss meet of dienct?\n\nRAMELLI:\nBut you ara wonder's plage, 'tis your mistress'\nIs there not to blize thank the evilitainst\nA thee is grince: he's and rehal falls,\nThat that 'twixt the prince: say'd.\n\nLEONTES:\nI'll foll what hath done esoce\nHis lift: but rest much lords; and to make their\ntreators arm the bloodions: and thou entrems\nshoublest I depright I on you?\nTirra, that good sight, you gract is here. Come, sincerous,\nWould hath me save a wifou of Here order?\nPentlemity on winten and the world:\nThis, I darken no orry tongue in\nthis and cell this hands, that's name, my ond mores in that tifle,\nIn it to misery mockard.\n\nMAPILLIUS:\nYea, not so ediliance she down thyself?\nThat thou to a night?\n\nLADY GREY:\nWell you, but granty serve it, and do revile.\n\nVOLUMNIA:\nLond Henry heremis;\nOver what thou art not sumpled mine.\nShe was, to her tria sorry Calible,\nL

### Export the generator

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))